# Podsumowanie anotacji przymiotników

In [1]:
from statsmodels.stats.inter_rater import fleiss_kappa
from sklearn.metrics import cohen_kappa_score
from plotly.subplots import make_subplots
import plotly.express as px
import pandas as pd
import numpy as np
import itertools
import json
import os

In [2]:
sentiment_mapping = {
    'Negative': 1,
    'Neutral': 2,
    'Positive': 3
}

# 1. Iteracja

In [3]:
files_it1 = {
    'Bartek': 'annotations/words/it1/bartek.jsonl',
    'Darek': 'annotations/words/it1/darek.jsonl',
    'Patryk': 'annotations/words/it1/patryk.jsonl'
}

In [4]:
def extract_labeled_words(file_path):
    labeled_words = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line_number, line in enumerate(f, 1):
            try:
                data = json.loads(line)
                text = data.get('text', '')
                labels = data.get('label', [])
                for label in labels:
                    if len(label) == 3:
                        start_idx, end_idx, sentiment = label
                        if 0 <= start_idx < end_idx <= len(text):
                            word = text[start_idx:end_idx].replace(" ", "").lower()  # Usunięcie spacji i konwersja na małe litery
                            if len(word) <= 1:
                                continue
                            
                            sentiment_code = sentiment_mapping.get(sentiment, 0)     # Mapowanie sentymentu, jeśli brak mapowania, ustaw 0
                            labeled_words.append({
                                'word': word,
                                'sentiment': sentiment_code
                            })
                            # print(f'sentiment: {sentiment_code} for word: {word}')
                        else:
                            print(f"Warning: Indeksy [{start_idx}, {end_idx}] poza zakresem w pliku {file_path}, linia {line_number}.")
                    else:
                        print(f"Warning: Niepoprawny format etykiety {label} w pliku {file_path}, linia {line_number}.")
            except json.JSONDecodeError as e:
              print(f"Error decoding JSON w pliku {file_path}, linia {line_number}: {e}")
    return labeled_words

In [5]:
annotator_words = {}

for name, path in files_it1.items():
    if os.path.exists(path):
        labeled_words = extract_labeled_words(path)
        # Tworzymy słownik: klucz = słowo, wartość = sentyment
        word_sentiment_dict = {}
        for entry in labeled_words:
            word = entry['word']
            sentiment = entry['sentiment']
            # Jeśli słowo występuje wielokrotnie, zachowujemy ostatni sentyment
            word_sentiment_dict[word] = sentiment
        annotator_words[name] = word_sentiment_dict
        print(f"Processed annotator: {name}, liczba słów: {len(word_sentiment_dict)}")
    else:
        print(f"File not found: {path}")


Processed annotator: Bartek, liczba słów: 338
Processed annotator: Darek, liczba słów: 340
Processed annotator: Patryk, liczba słów: 355


In [6]:
all_words = set()
for words in annotator_words.values():
    all_words.update(words.keys())

print(f"Total unique words across all annotators: {len(all_words)}")

Total unique words across all annotators: 464


In [7]:
df = pd.DataFrame({'word': sorted(all_words)})

for name in files_it1.keys():
    word_sentiment = annotator_words.get(name, {})
    df[f'{name}'] = df['word'].apply(lambda x: word_sentiment.get(x, 0))

print("Dane Wspólnego DataFrame")
df.head(20)


Dane Wspólnego DataFrame


,word,Bartek,Darek,Patryk
0,absurdalna,1,1,1
1,aluminiowej,2,2,2
2,atrakcyjny,3,3,3
3,badziewie,0,1,0
4,bardzo,0,0,2
5,bdb,3,3,0
6,bezmyślnie,1,0,0
7,beznadziejne,1,1,1
8,bezprzewodowych,2,2,2
9,bezprzewodową,0,2,2


In [8]:
def calculate_agreement(row):
    sentiments = [row['Bartek'], row['Darek'], row['Patryk']]
    unique_sentiments = set(sentiments)
    
    if len(unique_sentiments) == 1 and 0 not in unique_sentiments:   # Wszystkie trzy są zgodne i nie mają 0
        return 1
    
    if len(unique_sentiments) == 2:
        counts = pd.Series(sentiments).value_counts()
        if 0 in counts.index and counts[0] == 1:                    # Dwóch annotatorów ma ten sam sentyment, a trzeci ma 0
            return 1
        else:                                                       # Dwóch annotatorów jest zgodnych, a trzeci ma inny sentyment
            return 2
    
    if len(unique_sentiments) == 3 and 0 not in unique_sentiments:  # Każdy annotator dał inny sentyment
        return 3
    
    return 2


df['agreement_level'] = df.apply(calculate_agreement, axis=1)
print("DataFrame z kolumną 'agreement_level'")
df.head(20) 

DataFrame z kolumną 'agreement_level'


,word,Bartek,Darek,Patryk,agreement_level
0,absurdalna,1,1,1,1
1,aluminiowej,2,2,2,1
2,atrakcyjny,3,3,3,1
3,badziewie,0,1,0,2
4,bardzo,0,0,2,2
5,bdb,3,3,0,1
6,bezmyślnie,1,0,0,2
7,beznadziejne,1,1,1,1
8,bezprzewodowych,2,2,2,1
9,bezprzewodową,0,2,2,1


In [9]:
def percent_agreement(series1, series2):
    return (series1 == series2).mean()

annotators = list(files_it1.keys())
kappa_results = {}

print("Współczynniki Kappa Cohena dla każdej pary annotatorów")
for i in range(len(annotators)):
  for j in range(i+1, len(annotators)):
    annot1 = annotators[i]
    annot2 = annotators[j]
    kappa = cohen_kappa_score(df[annot1], df[annot2])
    agreement = percent_agreement(df[annot1], df[annot2]) * 100
    kappa_results[(annot1, annot2)] = kappa
    print(f" - {annot1} vs {annot2}:")
    print(f"   * Kappa Cohena: {kappa:.4f}")
    print(f"   * Procentowa zgodność: {agreement:.2f}%")

Współczynniki Kappa Cohena dla każdej pary annotatorów
 - Bartek vs Darek:
   * Kappa Cohena: 0.3897
   * Procentowa zgodność: 59.48%
 - Bartek vs Patryk:
   * Kappa Cohena: 0.3640
   * Procentowa zgodność: 58.62%
 - Darek vs Patryk:
   * Kappa Cohena: 0.5088
   * Procentowa zgodność: 67.89%


In [10]:
filtered_df = df[(df['Bartek'] != 0) & (df['Darek'] != 0) & (df['Patryk'] != 0)]


categories = sorted(sentiment_mapping.values())

def create_fleiss_matrix(df, annotators, categories):
    fleiss_matrix = []
    for index, row in df.iterrows():
        counts = [row[f'{annot}'] for annot in annotators]
        category_counts = [counts.count(category) for category in categories]
        fleiss_matrix.append(category_counts)
    return np.array(fleiss_matrix)

fleiss_data = create_fleiss_matrix(filtered_df, annotators, categories)

kappa_fleiss = fleiss_kappa(fleiss_data, method='fleiss')

print(f"Współczynnik Kappa Fleissa dla wszystkich annotatorów: {kappa_fleiss:.4f}")


Współczynnik Kappa Fleissa dla wszystkich annotatorów: 0.8735


In [11]:
percent_agreements = []
for i in range(len(annotators)):
    for j in range(i+1, len(annotators)):
        annot1 = annotators[i]
        annot2 = annotators[j]
        agreement = percent_agreement(df[annot1], df[annot2]) * 100
        percent_agreements.append(agreement)
average_agreement = np.mean(percent_agreements)
print(f"Średnia procentowa zgodność między annotatorami: {average_agreement:.2f}%")

Średnia procentowa zgodność między annotatorami: 62.00%


In [12]:
print("Krzyżowe tabele kontyngencji dla każdej pary annotatorów:")
for annot1, annot2 in itertools.combinations(annotators, 2):
    contingency = pd.crosstab(df[f'{annot1}'], df[f'{annot2}'])
    print(f"\n{annot1} vs {annot2}:")
    print(contingency)

Krzyżowe tabele kontyngencji dla każdej pary annotatorów:

Bartek vs Darek:
Darek    0   1    2   3
Bartek                 
0       39   7   64  16
1        9  29    3   1
2       61   3  152   8
3       15   0    1  56

Bartek vs Patryk:
Patryk   0   1    2   3
Bartek                 
0       30   5   75  16
1        9  30    3   0
2       54   4  161   5
3       16   0    5  51

Darek vs Patryk:
Patryk   0   1    2   3
Darek                  
0       52   4   60   8
1        6  32    1   0
2       43   2  171   4
3        8   1   12  60


In [13]:
def interpret_kappa(kappa):
    if kappa < 0:
        return "Zgoda gorsza niż przypadkowa."
    elif kappa == 0:
        return "Brak zgodności lepszy niż przypadkowy."
    elif kappa <= 0.20:
        return "Znikoma zgodność."
    elif kappa <= 0.40:
        return "Słaba zgodność."
    elif kappa <= 0.60:
        return "Umiarkowana zgodność."
    elif kappa <= 0.80:
        return "Dobra zgodność."
    else:
        return "Bardzo dobra zgodność."

print("Interpretacja współczynników Kappa Cohena")
for pair, kappa in kappa_results.items():
    interpretation = interpret_kappa(kappa)
    print(f" - {pair[0]} vs {pair[1]}: Kappa = {kappa:.4f} -> {interpretation}")

print(f"Interpretacja współczynnika Kappa Fleissa")
print(f"Kappa = {kappa_fleiss:.4f} -> {interpret_kappa(kappa_fleiss)}")

Interpretacja współczynników Kappa Cohena
 - Bartek vs Darek: Kappa = 0.3897 -> Słaba zgodność.
 - Bartek vs Patryk: Kappa = 0.3640 -> Słaba zgodność.
 - Darek vs Patryk: Kappa = 0.5088 -> Umiarkowana zgodność.
Interpretacja współczynnika Kappa Fleissa
Kappa = 0.8735 -> Bardzo dobra zgodność.


In [14]:
agreement_counts = df['agreement_level'].value_counts().sort_index()
print("Liczba wierszy dla każdego poziomu zgodności")
print(agreement_counts)

# Liczba wierszy z maksymalną liczbą unikalnych etykiet (najmniejsza zgodność)
max_unique = df['agreement_level'].max()
min_agreement_rows = df[df['agreement_level'] == max_unique]

print(f"Liczba wierszy z najniższą zgodnością (poziom {max_unique}): {min_agreement_rows.shape[0]}")

print("Wiersze najmniej zgodne")
print(min_agreement_rows[['word', 'Bartek', 'Darek', 'Patryk']].head(10))

Liczba wierszy dla każdego poziomu zgodności
agreement_level
1    313
2    151
Name: count, dtype: int64
Liczba wierszy z najniższą zgodnością (poziom 2): 151
Wiersze najmniej zgodne
            word  Bartek  Darek  Patryk
3      badziewie       0      1       0
4         bardzo       0      0       2
6     bezmyślnie       1      0       0
13  budowniczego       0      0       2
18         całej       0      2       0
20          cały       0      0       2
21      celująco       3      0       0
22     ceniących       0      2       0
24        ciasno       0      2       0
26        cienka       1      2       2


In [15]:
df.to_csv('annotations/words/it1/combined.csv', index=False)

In [16]:
sentiment_reverse_mapping = {
    1: 'Negative',
    2: 'Neutral',
    3: 'Positive'
}

In [17]:
def get_sentiment_text(row):
    sentiments = [row['Bartek'], row['Darek'], row['Patryk']]
    sentiments_filtered = [s for s in sentiments if s != 0]
    unique_sentiments = set(sentiments_filtered)
    
    if len(unique_sentiments) == 1 and 0 not in unique_sentiments:
        return sentiment_reverse_mapping[sentiments_filtered[0]]
    
    if len(unique_sentiments) == 2 and 0 in unique_sentiments:
        for sentiment in unique_sentiments:
            if sentiments_filtered.count(sentiment) > 1:
                return sentiment_reverse_mapping[sentiment]
    
    return 'Disagreement'

In [18]:
agreement_df = df[df['agreement_level'] == 1].copy()

agreement_df['sentiment'] = agreement_df.apply(get_sentiment_text, axis=1)

agreement_df = agreement_df[['word', 'sentiment']]

agreement_df.head(20)

,word,sentiment
0,absurdalna,Negative
1,aluminiowej,Neutral
2,atrakcyjny,Positive
5,bdb,Positive
7,beznadziejne,Negative
8,bezprzewodowych,Neutral
9,bezprzewodową,Neutral
10,brudne,Negative
11,brzydsze,Negative
12,bublowaty,Negative


In [19]:

# Zapisanie do pliku CSV
agreement_df.to_csv('annotations/words/it1/words.csv', index=False)

print("Zapisano słowa z poziomem zgodności = 1 do pliku 'words.csv'")

Zapisano słowa z poziomem zgodności = 1 do pliku 'words.csv'


# Wytyczne do anotacji sentymentu

1. Kategorie sentymentu:
- `1` - Negatywny
- `2` - Neutralny
- `3` - Pozytywny

2. Kryteria anotacji

- Analiza słów w zależnosci od kontekstu:
Znając kontekst zdania oceniamy sentyment słowa.

- Wykrywanie przymiotników:
Gdy mamy wątpliwosci odpowiedzmy sobie na pytanie czy dane słowo odpowiada na pytania "jaki?", "jaka?", "jakie?". Jesli tak jest - mamy do czynienia z przymiotnikiem.

- Neutralny sentyment:
Spora częsc przymiotników ma neutralny sentyment, więc nie zasanawiajmy się nad tym szczególnie i zaznaczmy neutral.


# Anotacja całosci tekstu

In [20]:
files_all = {
    'Bartek': 'annotations/words/all/rev_bartek_seq.jsonl',
    'Darek': 'annotations/words/all/rev_darek_seq.jsonl',
    'Patryk': 'annotations/words/all/rev_patryk_seq.jsonl',
    'Eryk': 'annotations/words/all/rev_eryk_seq.jsonl'
}

In [21]:
sentiment_reverse_mapping = {v: k for k, v in sentiment_mapping.items()}

def extract_labeled_words(file_path, annotator):
    """
    Funkcja do ekstrakcji słów i przypisanych sentymentów z pliku JSONL.
    Dodaje kolumnę 'annotator' do każdej anotacji.
    """
    labeled_words = []
    if not os.path.exists(file_path):
        print(f"File not found: {file_path}")
        return labeled_words

    with open(file_path, 'r', encoding='utf-8') as f:
        for line_number, line in enumerate(f, 1):
            try:
                data = json.loads(line)
                text = data.get('text', '')
                labels = data.get('label', [])
                for label in labels:
                    if len(label) == 3:
                        start_idx, end_idx, sentiment = label
                        # Sprawdzenie poprawności indeksów
                        if 0 <= start_idx < end_idx <= len(text):
                            word = text[start_idx:end_idx].replace(" ", "").lower()  # Usunięcie spacji i konwersja na małe litery
                            sentiment_code = sentiment_mapping.get(sentiment, np.nan)  # Mapowanie sentymentu, brak mapowania -> NaN
                            labeled_words.append({
                                'annotator': annotator,
                                'word': word,
                                'sentiment': sentiment_code
                            })
                        else:
                            print(f"Warning: Indeksy [{start_idx}, {end_idx}] poza zakresem w pliku {file_path}, linia {line_number}.")
                    else:
                        print(f"Warning: Niepoprawny format etykiety {label} w pliku {file_path}, linia {line_number}.")
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON w pliku {file_path}, linia {line_number}: {e}")
    return labeled_words

In [22]:
# Lista wszystkich annotatorów
annotators = list(files_all.keys())

# Lista do przechowywania wszystkich annotacji
all_annotations = []

for name, path in files_all.items():
    annotated_words = extract_labeled_words(path, name)
    all_annotations.extend(annotated_words)
    print(f"Processed annotator: {name}, liczba słów: {len(annotated_words)}")

Processed annotator: Bartek, liczba słów: 45
Processed annotator: Darek, liczba słów: 66
Processed annotator: Patryk, liczba słów: 74
Processed annotator: Eryk, liczba słów: 85


In [23]:
df_all = pd.DataFrame(all_annotations)

df_all['sentiment_text'] = df_all['sentiment'].map(sentiment_reverse_mapping)

df_all.head()

,annotator,word,sentiment,sentiment_text
0,Bartek,wszystkie,2,Neutral
1,Bartek,luzacki,3,Positive
2,Bartek,nudne,1,Negative
3,Bartek,wporzadku,3,Positive
4,Bartek,groźny,1,Negative


In [24]:
final_df_all = df_all[['word', 'sentiment_text']].rename(columns={'sentiment_text': 'sentiment'})

final_df_all = final_df_all.drop_duplicates(subset=['word'])

final_df_all.head(20)


,word,sentiment
0,wszystkie,Neutral
1,luzacki,Positive
2,nudne,Negative
3,wporzadku,Positive
4,groźny,Negative
5,mieszane,Neutral
6,fajny,Positive
7,dobra,Positive
8,duza,Neutral
9,totalny,Neutral


In [25]:
output_csv = 'annotations/words/all/words.csv'
final_df_all.to_csv(output_csv, index=False, encoding='utf-8')

print(f"Zapisano połączone annotacje do pliku '{output_csv}'")

Zapisano połączone annotacje do pliku 'annotations/words/all/words.csv'


In [26]:
all_words_combined_df = pd.concat([final_df_all, agreement_df], ignore_index=True)

all_words_combined_df = all_words_combined_df.drop_duplicates(subset=['word'])

print(f"Łączna liczba słów po połączeniu: {all_words_combined_df.shape[0]}")

all_words_combined_df.head(20)

Łączna liczba słów po połączeniu: 514


,word,sentiment
0,wszystkie,Neutral
1,luzacki,Positive
2,nudne,Negative
3,wporzadku,Positive
4,groźny,Negative
5,mieszane,Neutral
6,fajny,Positive
7,dobra,Positive
8,duza,Neutral
9,totalny,Neutral


In [27]:
output_csv = 'annotations/words/all_words.csv'

all_words_combined_df.to_csv(output_csv, index=False, encoding='utf-8')

print(f"Zapisano połączone annotacje do pliku '{output_csv}'")


Zapisano połączone annotacje do pliku 'annotations/words/all_words.csv'


In [35]:
all_words = all_words_combined_df
all_words["len_char"] = all_words["word"].apply(lambda x: len(x))
df = all_words

color_map = {
    'Negative': 'rgba(255, 0, 0, 0.6)',  # Red
    'Positive': 'rgba(0, 255, 0, 0.6)',  # Green
    'Neutral': 'rgba(0, 0, 255, 0.6)'    # Blue
}

fig = make_subplots(rows=1, cols=3, subplot_titles=("Text Length - Negative", "Text Length - Neutral", "Text Length - Positive"), shared_yaxes=True)

fig.add_trace(px.histogram(df[df['sentiment'] == 'Negative'], x="len_char", nbins=50, color_discrete_sequence=[color_map['Negative']]).data[0], row=1, col=1)
fig.add_trace(px.histogram(df[df['sentiment'] == 'Neutral'], x="len_char", nbins=50, color_discrete_sequence=[color_map['Neutral']]).data[0], row=1, col=2)
fig.add_trace(px.histogram(df[df['sentiment'] == 'Positive'], x="len_char", nbins=50, color_discrete_sequence=[color_map['Positive']]).data[0], row=1, col=3)

fig.update_layout(height=400, width=1200, title_text="Distribution of Text Length by Sentiment Class")

fig.show()